<a href="https://colab.research.google.com/github/darkmochalover/DS_TermProject_AL2/blob/main/wandb_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dictionary

In [1]:
# Data Path
raw_data_path = "/content/drive/MyDrive/DS_TermProject/data/combined_mbti_df.csv"

audio_qualities = [
    'danceability_mean',
    'valence_mean',
    'energy_mean',
    'loudness_mean',
    'acousticness_mean',
    'instrumentalness_mean',
    'liveness_mean',
]

# 장조/단조 (Major/Minor)
all_tones = [
    'Cminor_count', 'CMajor_count', 'C#/Dbminor_count', 'C#/DbMajor_count',
    'DMajor_count', 'D#_EbMajor_count', 'Eminor_count', 'EMajor_count',
    'Fminor_count', 'FMajor_count', 'F#/Gbminor_count', 'GMajor_count',
    'G#/Abminor_count', 'G#/AbMajor_count', 'Aminor_count', 'AMajor_count',
    'A#/Bbminor_count', 'BMajor_count', 'Dminor_count', 'D#_Ebminor_count',
    'Gminor_count', 'A#/BbMajor_count', 'F#/GbMajor_count', 'Bminor_count'
]

major_tones = [
    'CMajor_count', 'C#/DbMajor_count',
    'DMajor_count', 'D#_EbMajor_count', 
    'EMajor_count',
    'FMajor_count', 
    'GMajor_count', 'G#/AbMajor_count', 
    'AMajor_count', 'BMajor_count', 'A#/BbMajor_count', 
    'F#/GbMajor_count'
]
minor_tones = [
    'Cminor_count', 'C#/Dbminor_count', 
    'Eminor_count', 
    'Fminor_count', 'F#/Gbminor_count', 
    'G#/Abminor_count',  
    'Aminor_count', 'A#/Bbminor_count', 
    'Dminor_count', 'D#_Ebminor_count',
    'Gminor_count', 
    'Bminor_count'
]


# 열 이름을 바꾸기 위해 리스트로 저장함
renamed_columns =  [
    'danceability',
    'valence',
    'energy',
    'loudness',
    'acousticness',
    'instrumentalness',
    'liveness'
]


Install WandB

In [2]:
!pip install -qU wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 61.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.5/206.5 kB 22.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.8 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from imblearn.over_sampling import SMOTE

import itertools

import wandb
import random

Step 1: Import W&B and Login

In [4]:
import wandb
from wandb.keras import WandbMetricsLogger, WandbModelCheckpoint, WandbEvalCallback

In [5]:
encoder_list = ['LabelEncoder', 'OneHotEncoder']
scaler_list = ['StandardScaler', 'MinMaxScaler', 'RobustScaler', 'No Scale']
test_size_list = [0.3, 0.2, 0.1]
do_smote = ['True', 'False']

In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

combinations = list(itertools.product(encoder_list, scaler_list, test_size_list, do_smote))



Data Loading

In [7]:
df = pd.read_csv(raw_data_path)

Feature Reduction

In [8]:
# Subset only measures of centers
X = df.iloc[: , :22] # 처음 22개 열을 선택해서 저장
X = df[audio_qualities] # 오디오 품질과 해당 열이 있는 열을 선택해서 저장


categories = renamed_columns[:]
X.columns = renamed_columns


# 장조/단조의 개수의 합을 계산해서 저장 (C장조, D단조, .. 이렇게 따로 계산되는거 말고, 위에 지정된 list 이용해서 sum값 넣어줌)
X['major_count'] = df[major_tones].sum(axis=1).astype('int64')
X['minor_count'] = df[minor_tones].sum(axis=1).astype('int64')

<ipython-input-8-482ca4c8bc7c>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['major_count'] = df[major_tones].sum(axis=1).astype('int64')
<ipython-input-8-482ca4c8bc7c>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['minor_count'] = df[minor_tones].sum(axis=1).astype('int64')


## Encoding & Scaling Function

In [9]:
def Encoding(df, encoding_method):
    df = df.copy()

    if(encoding_method == 'LabelEncoder'):
        encoder = LabelEncoder()
        target = encoder.fit_transform(df[['mbti']])
        

    if(encoding_method == 'OneHotEncoder'):
        encoder = OneHotEncoder(sparse=False)
        target = encoder.fit_transform(df[['mbti']])

    return target

def Scaling(scale_method, X_train, X_test):
    if( scale_method == 'No Scale'):
        return X_train, X_test

    elif(scale_method == 'StandardScaler'):
        scaler = StandardScaler()

    elif(scale_method == 'MinMaxScaler'):
        scaler = MinMaxScaler()

    elif(scale_method == 'RobustScaler'):
        scaler = RobustScaler()


    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.fit_transform(X_test)

    return X_train_scaled, X_test_scaled


바꿔가면서 돌리기

In [10]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [11]:
import shutil
# os.rmdir('/content/wandb')
shutil.rmtree('/content/wandb', ignore_errors=True)

In [ ]:
for encoder, scaler, test_size, do_smote in combinations:
  wandb.init(project="MBTI_testing")

  wandb.config = {
            'encoder' : encoder,
            'scaler' : scaler,
            'test_size' : test_size,
            "do_smote" : do_smote
  }

  config = wandb.config
  print(config)

  y = Encoding(df = df, encoding_method=encoder)
  # print(y[:5])

  X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 5, test_size = test_size)

  X_train, X_test = Scaling(scale_method=scaler, X_train=X_train, X_test=X_test)

  if(do_smote == 'True'): # If true,
      smote = SMOTE(sampling_strategy='auto', random_state=0)
      X_train, y_train = smote.fit_resample(X_train,y_train)

  print('Train 피처/레이블 데이터 세트: ', X_train.shape, y_train.shape)

  # wandb.log({'Train feature 데이터 세트' : X_train.shape})
  # wandb.log({'Train label 데이터 세트' : X_train.shape})

  # Random Forest Model with top 5 features
  model = RandomForestClassifier(random_state=42)
  model.fit(X_train, y_train)
  # prediction = model.predict(X_test)
  # acc = accuracy_score(y_test, prediction)
  # print("Random Forest Accuracy with Top 5 Features:", acc)
  # wandb.log({'Random Forest Accuracy with Top 5 Features' : acc})

  # 평가
  train_acc = model.score(X_train, y_train)
  test_acc = model.score(X_test, y_test)

  print("훈련 세트 정확도: {:.3f}".format(train_acc) )
  print("테스트 세트 정확도: {:.3f}".format(test_acc) )

  # acc 추적
  wandb.define_metric('train_acc', summary='max')
  wandb.define_metric('test_acc', summary='max')
  
  wandb.log({'train_acc' : train_acc })
  wandb.log({'test_acc' : test_acc })

  # # loss 추적
  # wandb.define_metric('train_loss', summary='min')
  # wandb.define_metric('val_loss', summary='min')

  # # f1 score 추적
  # wandb.define_metric('train_f1', summary='max')
  # wandb.define_metric('val_f1', summary='max')

  


        


test_acc,▇▇▇▇▇█▆▆▅▆▆▆▇▇█▇▇█▇▇▇▇▇█▁▁▁
train_acc,█▆█▆▇▅█▆█▆▇▅█▆█▆▇▅█▆█▆▇▅▄▁▃


{'encoder': 'LabelEncoder', 'scaler': 'StandardScaler', 'test_size': 0.3, 'do_smote': 'True'}
Train 피처/레이블 데이터 세트:  (3456, 9) (3456,)


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


훈련 세트 정확도: 0.974
테스트 세트 정확도: 0.239


test_acc,▁
train_acc,▁


...

훈련 세트 정확도: 0.963
테스트 세트 정확도: 0.267


test_acc,▁
train_acc,▁


{'encoder': 'LabelEncoder', 'scaler': 'No Scale', 'test_size': 0.3, 'do_smote': 'True'}
Train 피처/레이블 데이터 세트:  (3456, 9) (3456,)


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


훈련 세트 정확도: 0.974
테스트 세트 정확도: 0.220


In [ ]:
wandb.finish()